In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 光纤布拉格光栅参数
neff = 1.45  # 有效折射率
kappa = 10   # 光栅强度参数，假定一个值
L = 10e-3    # 光栅长度

# 布拉格波长
lambdaB = 2 * neff * 1e-3  # 这里的布拉格波长是假定的，用户需要根据实际情况设置

# 波长范围
lambda_range = np.linspace(1540e-9, 1560e-9, 1000)  # 从1540nm到1560nm

# 计算反射率
R = np.zeros_like(lambda_range)  # 反射率数组初始化

# 遍历每一个波长并计算反射率
for idx, lambda_ in enumerate(lambda_range):
    delta = 2 * np.pi * neff / lambda_ - np.pi / lambdaB
    s = np.sqrt(kappa**2 - delta**2 + 0j)  # 添加小的虚部以避免在delta接近kappa时开根号成为负数

    # 传输矩阵的元素
    S11 = np.cosh(s * L) + (1j * delta / s) * np.sinh(s * L)
    S12 = (1j / kappa) * np.sinh(s * L)
    S21 = S12
    S22 = S11

    # 传输矩阵
    F = np.array([[S11, S12], [S21, S22]])

    # 入射场和反射场
    A = np.array([1, 0])  # 假设入射场为1，反射场为0

    # 通过传输矩阵计算输出场
    B = np.linalg.solve(F, A)

    # 计算反射率
    R[idx] = np.abs(B[1])**2

# 绘图
plt.figure(figsize=(10, 6))
plt.plot(lambda_range * 1e9, R, label='Reflectivity')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectivity')
plt.title('Reflectivity of a Uniform Fiber Bragg Grating')
plt.legend()
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [6]:
import fitz  # PyMuPDF
from pptx import Presentation
from pptx.util import Inches

def pdf_to_ppt(pdf_path, ppt_path):
    # 打开PDF文件
    doc = fitz.open(pdf_path)
    
    # 获取PDF页面尺寸（以英寸为单位）
    pdf_width = doc[0].rect[2] / 72
    pdf_height = doc[0].rect[3] / 72
    
    # 创建一个PPT文件，设置尺寸为PDF页面尺寸
    pres = Presentation()
    pres.slide_width = Inches(pdf_width)
    pres.slide_height = Inches(pdf_height)
    
    # 遍历PDF中的每一页
    for page_number in range(doc.page_count):
        page = doc[page_number]
        
        # 获取页面的尺寸（以点为单位）
        page_width = page.rect[2] - page.rect[0]
        page_height = page.rect[3] - page.rect[1]
        
        # 计算页面的缩放比例
        # scale_x = pdf_width / (page_width / 72)
        # scale_y = pdf_height / (page_height / 72)
        
        # 将页面渲染为图像
        zoom_x = 10 # horizontal zoom
        zomm_y = 10 # vertical zoom
        mat = fitz.Matrix(zoom_x, zomm_y) # zoom factor 2 in each dimension
        pix = page.get_pixmap(matrix=fitz.Matrix(zoom_x, zomm_y))
        
        # 将图像保存到临时文件
        img_path = f"page_{page_number}.png"
        pix.save(img_path)
        
        # 创建一个新的幻灯片
        slide = pres.slides.add_slide(pres.slide_layouts[5])
        
        # 在幻灯片中添加图像，并设置位置和大小
        slide.shapes.add_picture(img_path, Inches(0), Inches(0), width=Inches(pdf_width), height=Inches(pdf_height))
        
        # 删除临时保存的图像文件
        os.remove(img_path)
    
    # 保存PPT文件
    pres.save(ppt_path)

# 指定PDF和PPT的路径
pdf_path = '研讨.pdf'
ppt_path = 'output1.pptx'

# 调用函数
pdf_to_ppt(pdf_path, ppt_path)
